In [1]:
from FM_solution import *

## 가상의 주식 테이블 생성하기
- 시점 $T$ 는 1부터 1000까지 생성
- 개별 종목의 갯수는 400개로 설정
- 난수 발생으로 임의로 정하였다.
--------------------------------------------
## Notice : 데이터 포맷 그 이상의 의미를 가지지 못한다.

In [2]:
code_df = pd.read_csv('industrial_code.csv')
code_df.rename(columns={'종목코드':'code'},inplace=True)
code_df = code_df[:401]
num_of_stock = 400
T = 1000
df = pd.DataFrame(columns=[i for i in range(1,T+1)])

for i in df.columns:
    df[i] = [0 for i in range(num_of_stock + 1)]
    
df.index = list(code_df['code'])
df.index = [str(i) for i in df.index]

for i in range(len(df)):
    df.iloc[i] = [(np.random.randint(10))*(1+np.random.rand()) for _ in range(1000)]
    
df.index = [str(i) for i in df.index]
df = df.append(pd.Series([1 for _ in range(T)], index=df.columns), ignore_index=True) 
df.rename(index={df.index[-1]:'bias'},inplace=True)
df.rename(index={df.index[-2]:'market'},inplace=True)
individual_stock_df = df[(df.index != 'market') & (df.index != 'bias')]
df.index = list(code_df['code'])[:-1] + ['market','bias']
display(df.tail())
print(df.shape)

,1,2,3,4,5,6,7,8,9,10,...,991,992,993,994,995,996,997,998,999,1000
17810,1.257295,10.773807,13.946398,6.906598,7.789312,1.510906,9.241267,0.000000,10.969686,3.226850,...,10.716502,15.980701,5.891332,8.369528,8.103619,12.197811,0.000000,15.678048,4.773781,6.489053
11200,2.778111,9.814250,13.994607,3.154554,0.000000,10.191331,11.147793,12.719244,7.573940,1.184633,...,8.323932,9.433183,8.869402,5.861731,11.202679,7.389168,16.453325,11.764520,11.923806,11.656796
17390,5.793845,0.000000,11.358391,1.733969,3.437534,2.277374,3.587492,6.882197,3.737850,15.312050,...,1.951352,6.577574,8.829078,7.510783,7.512166,6.684229,9.539525,0.000000,11.026651,4.454646
market,9.332893,9.342022,13.605822,14.757272,7.862481,6.908221,6.630333,13.247908,5.072134,8.517387,...,4.711660,2.853174,8.650822,7.648198,12.007319,12.286149,1.588059,1.643908,10.901972,9.129998
bias,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


(402, 1000)


## 1단계 : 포트폴리오 구성시기 (portfolio formation period)
- 시장모형의 시계열회귀분석으로 개별 종목에 대한 $\beta$ 계수를 추정한다.
- 개별 종목에 대한 $\beta$ 400개 계수 측정이 완료되면, 크기 순으로 나열해서 20개씩 묶어, 포트폴리오를 형성한다.
    - 개별 종목의 갯수를 400개로 한 이유가 이와 같다.

In [3]:
testing_beta , testing_resid = first_OLS(individual_stock_df,df)

시간 소요가 걸리는 작업입니다. 잠시만 기다려주세요!

In [4]:
len(testing_beta) , len(testing_resid)

(400, 400)

In [5]:
beta , resid , stock , idx , testing_df = first_sorting_beta_resid(testing_beta,testing_resid,df)

In [6]:
len(beta) , len(resid) , len(stock) , len(idx)

(20, 20, 20, 20)

In [7]:
testing_df.tail()

,beta,resid,valued_stock_idx,valued_stock
15,0.662555,5.572278,"[153, 171, 40, 318, 167, 227, 301, 320, 347, 7...","[123690, 82640, 241590, 51630, 114090, 9770, 1..."
16,0.658517,5.502634,"[50, 330, 292, 103, 355, 173, 328, 113, 72, 85...","[70960, 9540, 36530, 155900, 12320, 93240, 365..."
17,0.652683,5.600403,"[262, 51, 389, 264, 164, 116, 138, 279, 297, 2...","[41650, 101530, 15890, 23810, 12160, 23000, 58..."
18,0.643930,5.549179,"[251, 363, 349, 74, 139, 281, 162, 169, 283, 2...","[84690, 29530, 10600, 112610, 130660, 72130, 7..."
19,0.627285,5.546118,"[244, 147, 114, 380, 325, 92, 213, 314, 24, 38...","[1560, 122900, 140890, 910, 36460, 155660, 972..."


# 2단계 : 포트폴리오 $\beta$ 의 추정시기 (estimation period)

- 시장 모형의 시계열회귀분석으로 20개 포트폴리오에 대한 $\beta$ 계수를 추정한다.
- 개별 종목에 대한 $\beta_i$ 를 구하고, 포트폴리오를 형성, 시계열 평균으로 $\beta_P$ 를 추정한다.

In [8]:
testing_beta2,testing_resid2 = second_OLS(individual_stock_df,df)

시간 소요가 걸리는 작업입니다. 잠시만 기다려주세요!

In [9]:
len(testing_beta2) , len(testing_resid2)

(400, 400)

In [10]:
final_beta , final_resid , final_df = second_sorting_beta_resid(testing_beta2,testing_resid2,df,testing_df)

In [11]:
len(final_beta) , len(final_resid)

(20, 20)

In [12]:
final_df.tail()

,beta,resid,stock_idx,stock
15,0.706114,5.588412,"[153, 171, 40, 318, 167, 227, 301, 320, 347, 7...","[123690, 82640, 241590, 51630, 114090, 9770, 1..."
16,0.703023,5.468613,"[50, 330, 292, 103, 355, 173, 328, 113, 72, 85...","[70960, 9540, 36530, 155900, 12320, 93240, 365..."
17,0.697742,5.543084,"[262, 51, 389, 264, 164, 116, 138, 279, 297, 2...","[41650, 101530, 15890, 23810, 12160, 23000, 58..."
18,0.690196,5.520127,"[251, 363, 349, 74, 139, 281, 162, 169, 283, 2...","[84690, 29530, 10600, 112610, 130660, 72130, 7..."
19,0.691030,5.506548,"[244, 147, 114, 380, 325, 92, 213, 314, 24, 38...","[1560, 122900, 140890, 910, 36460, 155660, 972..."


# 3단계 : 검증시기(testing period)
- 앞의 2단계에 의한여 20개의 포트폴리오의 베타 $\beta_P$ 가 추정될 것이므로 이 $\beta_P$ 와 포트폴리오에 속한 개별증권의 1개월간의 기대수익률과의 횡단면 회귀분석을 하여 식의 회귀 계수를 각각 추정한다.

In [13]:
nth_term = 1
testing_stock_df = individual_stock_df.iloc[:,900+(10*(nth_term-1)):900+(10*nth_term)]
testing_stock_df[185:200]

,901,902,903,904,905,906,907,908,909,910
185,11.967879,11.774982,10.071176,7.533462,7.728940,3.602533,5.530832,6.419041,9.511893,1.963959
186,12.883135,0.000000,8.137417,11.695998,4.257200,1.842159,4.822575,11.694451,6.004806,3.353291
187,0.000000,9.518520,3.326556,0.000000,4.664655,0.000000,5.222376,5.523833,2.370423,14.634652
188,12.756740,0.000000,9.274103,5.284612,10.305678,10.387905,0.000000,8.115199,11.326104,10.001697
189,10.681035,13.820821,3.562392,12.963876,13.838146,9.769842,0.000000,11.118017,11.702473,5.077049
190,4.484559,1.775012,14.843227,1.150343,9.656230,7.133692,3.421221,1.229679,0.000000,7.780107
191,5.762777,0.000000,0.000000,5.057349,6.484439,9.303933,0.000000,15.372440,7.368422,5.375626
192,2.692774,4.186476,0.000000,0.000000,8.726698,13.877861,7.151915,1.855598,10.437940,3.739577
193,8.299735,3.536852,0.000000,12.700555,6.473313,12.463782,1.184150,2.277416,2.680007,3.655677
194,3.376564,1.342395,7.662635,9.467132,2.051625,5.060845,4.044048,1.048067,9.459838,8.019744


In [17]:
gamma_1 , gamma_2 , gamma_3 , gamma_4 = real_df(individual_stock_df,final_df,nth_term=1)

In [18]:
gamma_1 , gamma_2 , gamma_3 , gamma_4

([-1955.8121149970034],
 [5793.2224269967001],
 [-4167.9655089252146],
 [-9.0542818149508975])

# 2단계,3단계 진행
- Time batch size 를 10으로 하고 window shift를 진행한다.
- shift를 할 때마다 $\beta_P$ 를 최신화하고, cross_sectional regression을 시행한다.
- cross_sectional regression 을 할 때마다, $\gamma_1$ , $\gamma_2$ , $\gamma_3$ , $\gamma_4$ 가 새로 생긴다.
- $\gamma_1$ , $\gamma_2$ , $\gamma_3$ , $\gamma_4$ 계수들이 시계열 $T$ 만큼 생성된다.
- $\gamma_1$ , $\gamma_2$ , $\gamma_3$ , $\gamma_4$ 의 시계열 평균값을 구하고, 실증가설을 T-test에 따라 검증한다.

In [35]:
%%time
real_gamma_1 , real_gamma_2 , real_gamma_3 , real_gamma_4 = [],[],[],[]

for i in range(10):
    testing_beta2,testing_resid2 = second_OLS(individual_stock_df,df,i=i)
    final_beta , final_resid , final_df = second_sorting_beta_resid(testing_beta2,testing_resid2,df,testing_df)
    gamma_1 , gamma_2 , gamma_3 , gamma_4 = real_df(individual_stock_df,final_df,nth_term=i)
    real_gamma_1.append(gamma_1)
    real_gamma_2.append(gamma_2)
    real_gamma_3.append(gamma_3)
    real_gamma_4.append(gamma_4)

시간 소요가 걸리는 작업입니다. 잠시만 기다려주세요!

시간 소요가 걸리는 작업입니다. 잠시만 기다려주세요!

시간 소요가 걸리는 작업입니다. 잠시만 기다려주세요!

시간 소요가 걸리는 작업입니다. 잠시만 기다려주세요!

시간 소요가 걸리는 작업입니다. 잠시만 기다려주세요!

...

시간 소요가 걸리는 작업입니다. 잠시만 기다려주세요!

시간 소요가 걸리는 작업입니다. 잠시만 기다려주세요!

시간 소요가 걸리는 작업입니다. 잠시만 기다려주세요!

시간 소요가 걸리는 작업입니다. 잠시만 기다려주세요!

CPU times: user 14min 13s, sys: 6.54 s, total: 14min 19s
Wall time: 3min 35s


In [61]:
display(Markdown("### $\gamma_1$ coef's t-test : {}".format(sp.stats.ttest_1samp(real_gamma_1,popmean=0)[1][0])))
display(Markdown("### $\gamma_2$ coef's t-test : {}".format(sp.stats.ttest_1samp(real_gamma_2,popmean=0)[1][0])))
display(Markdown("### $\gamma_3$ coef's t-test : {}".format(sp.stats.ttest_1samp(real_gamma_3,popmean=0)[1][0])))
display(Markdown("### $\gamma_4$ coef's t-test : {}".format(sp.stats.ttest_1samp(real_gamma_4,popmean=0)[1][0])))

### $\gamma_1$ coef's t-test : 0.8760835094847901

### $\gamma_2$ coef's t-test : 0.8755904542124714

### $\gamma_3$ coef's t-test : 0.878410476884093

### $\gamma_4$ coef's t-test : 0.34650087787477635